In [1]:
import findspark
findspark.init()

import pandas as pd
pd.set_option('display.max_rows', 1000)

In [2]:
import pyspark.sql.functions as F

In [3]:
from etl import SparkETL

In [4]:
etl = SparkETL()

In [5]:
spark = etl.get_spark()

22/05/14 17:56:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
flight = etl.read_fact_table('flight_fact')

In [7]:
from dim import TimeDim, VisitorDim, RouteDim, Dim

In [8]:
time_dim = TimeDim()
visitor_helper = VisitorDim()
route_helper = RouteDim()

In [9]:
def group(df, cols):
    return (
        df
        .groupby(*cols)
        .agg(
            F.sum('num_visitors').alias('count')
        )
    )

In [10]:
def join_dim(df, name, pk, cols=None):
    keys = cols if cols else [pk]
    dim = etl.read_dim_table(name)
    return (
        df.join(dim, on=Dim.on(df, dim, keys), how='inner')
    )    

In [11]:
def project_desc(df):
    return df.select(
        'count', 
        'citizenship', 
        'residence', 
        'age', 
        'gender', 
        'visa', 
        'address_state', 
        'address_type', 
        'address_climate', 
        'stay')

In [12]:
def plot(df, col):
    display(
        df
        .limit(20)
        .toPandas()
        .plot(x=col, y='count', kind='barh')
    )
    return df

In [13]:
def scatter(df):
    df.toPandas().plot(x=cols[-1], y='count', kind='scatter')

In [14]:
def sort_count(df, cols):
    return df.sort(*cols, F.desc('count'))

In [27]:
def sort(df, cols):
    return df.sort(*cols)

In [15]:
def where(df, cond):
    return df.where(cond)# if cond else df

In [33]:
def window(df):
    return df.withColumn('r', F.expr("""
        rank() over (partition by dst_state_name order by count desc) as rank
    """))

In [34]:
cols = ['dst_state_name', 'residence']

In [35]:
cond = F.col('residence') != F.col('citizenship')

In [38]:
(
    flight
    #.pipe(join_dim, 'time_dim', 'time_id')
    .pipe(join_dim, 'route_dim', 'route_id')
    .pipe(join_dim, 'foreign_visitor_dim', 'visitor_id')
    #.pipe(where, cond)
    #.pipe(project_desc)
    .pipe(group, cols)
    #.pipe(sort_count, ['residence'])
    .pipe(window)
    .pipe(where, F.expr("rank == 1"))
    .sort(['dst_state_name'])
    #.pipe(plot, cols[1])
    #.pipe(scatter)
    .pipe(lambda df: display(df.limit(1000).toPandas()))    
);

,dst_state_name,residence,count,r
0,None,JAPAN,769316,1
1,Alabama,COLOMBIA,27,1
2,Alaska,GERMANY,4823,1
3,Arizona,MEXICO,51133,1
4,California,"CHINA, PRC",1044712,1
5,Colorado,MEXICO,37516,1
6,Connecticut,UNITED KINGDOM,875,1
7,Delaware,GERMANY,14,1
8,District of Columbia,UNITED KINGDOM,128682,1
9,Florida,UNITED KINGDOM,1094175,1


In [30]:
etl.read_clean_table('immigration').count()

39166088

In [32]:
flight.agg(F.sum('num_visitors')).show()

+-----------------+
|sum(num_visitors)|
+-----------------+
|         39166088|
+-----------------+

